In [1]:
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# utils
from scipy.misc import imread
from scipy.misc import imresize
import os
import random

# Configure visualisations
%matplotlib inline
color = sns.color_palette()
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )

In [2]:
def bar_plot(df, col, text = True, k = None, rot = 0):
    """
    Arguments:
    x : pandas series
    Returns:
    seaborn barplot with text on top
    """
    if k == None:
        x = df[col].value_counts()
    else:
        x = df[col].value_counts()[:k]
    ax= sns.barplot(x.index, x.values, alpha=0.8)
    plt.title("# per class")
    plt.ylabel('# of Occurrences', fontsize=12)
    plt.xlabel(col, fontsize=12)
    #adding the text labels
    if text:
        rects = ax.patches
        labels = x.values
        for rect, label in zip(rects, labels):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
    plt.xticks(rotation = rot)
    plt.show()

In [3]:

# set the root path
root_dir = os.path.abspath('.')
# set the data dir
data_dir = '../input'

In [4]:
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [5]:
print("Train shape :",train.shape)
print("Test shape :",test.shape)

Train shape : (19906, 2)
Test shape : (6636, 1)


In [6]:
def resize_img(df, main_folder, folder_name):
    temp = []
    for index,img_name in enumerate(df.ID):
        if index % 1000 == 0:
            print ('Index :',index)
        img_path = os.path.join(data_dir, main_folder, folder_name, img_name)
        img = imread(img_path)
        img = imresize(img, (32, 32))
        img = img.astype('float32')
        temp.append(img)

    return np.stack(temp)

In [7]:
%%time
# resize all the images 32 x 32 (training set), usually takes ~ 4-5 min (CPU)
train_x = resize_img(train,"train" ,"Train")

Index : 0
Index : 1000
Index : 2000
Index : 3000
Index : 4000
Index : 5000
Index : 6000
Index : 7000
Index : 8000
Index : 9000
Index : 10000
Index : 11000
Index : 12000
Index : 13000
Index : 14000
Index : 15000
Index : 16000
Index : 17000
Index : 18000
Index : 19000
CPU times: user 15 s, sys: 1.85 s, total: 16.9 s
Wall time: 24.9 s


In [8]:
test_x = resize_img(test,"test" ,"Test")

Index : 0
Index : 1000
Index : 2000
Index : 3000
Index : 4000
Index : 5000
Index : 6000


In [9]:
train_x = train_x / 255
test_x = test_x / 255

In [10]:
import keras
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.np_utils.to_categorical(train_y)

Using TensorFlow backend.


In [11]:
input_num_units = (32, 32, 3)
hidden_num_units = 600
output_num_units = 3
num_filter = 100

epochs = 30
batch_size = 64

In [12]:

from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from keras.layers import Conv2D, MaxPooling2D, Dropout

In [13]:
model = Sequential()

## Input Layer
model.add(InputLayer(input_shape=input_num_units))

## Convolutional layer
model.add(Conv2D(num_filter, (3, 3), activation='relu', input_shape=input_num_units))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Conv2D(int(num_filter * 0.5), (3, 3), activation='relu', input_shape=(10, 10, 100)))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Conv2D(int(num_filter * 0.3), (3, 3), activation='relu', input_shape=(1, 1, 10)))
model.add(Flatten())
model.add(Dense(hidden_num_units, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(int(hidden_num_units * 0.2), activation='relu'))
model.add(Dense(units=output_num_units, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 100)       2800      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 100)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 100)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 50)        45050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 50)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 50)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 30)          13530     
__________

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1, validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 17915 samples, validate on 1991 samples
Epoch 1/30
17915/17915 [==============================] - 6s 345us/step - loss: 0.8937 - acc: 0.5814 - val_loss: 0.8258 - val_acc: 0.6238
Epoch 2/30
17915/17915 [==============================] - 2s 104us/step - loss: 0.8027 - acc: 0.6397 - val_loss: 0.7516 - val_acc: 0.6796
Epoch 3/30
17915/17915 [==============================] - 2s 105us/step - loss: 0.7523 - acc: 0.6691 - val_loss: 0.7111 - val_acc: 0.6951
Epoch 4/30
17915/17915 [==============================] - 2s 104us/step - loss: 0.7175 - acc: 0.6833 - val_loss: 0.6902 - val_acc: 0.7057
Epoch 5/30
17915/17915 [==============================] - 2s 103us/step - loss: 0.6943 - acc: 0.6941 - val_loss: 0.6854 - val_acc: 0.7102
Epoch 6/30
17915/17915 [==============================] - 2s 104us/step - loss: 0.6724 - acc: 0.7064 - val_loss: 0.6716 - val_acc: 0.7027
Epoch 7/30
17915/17915 [==============================] - 2s 105us/step - l

In [17]:
pred_train = model.predict_classes(train_x, verbose=1)
pred = model.predict_classes(test_x, verbose=1)
pred = lb.inverse_transform(pred)

test['Class'] = pred
test.to_csv('submission.csv', index=False)

6636/6636 [==============================] - 0s 49us/step
